In [1]:
import pandas as pd 

In [2]:
dfs = []
dfs.append(pd.read_csv("pop_data/all_pop_data_rtf.csv"))
dfs.append(pd.read_csv("pop_data/all_pop_data_txt.csv"))

In [3]:
all_data = pd.concat(dfs)
pgs = pd.read_csv("old_csv_data/prison_groups_lookup_and_manual_classification.csv")
pgs = pgs.fillna(0)
all_data = all_data.merge(pgs[["prison_name_original","prison_name_group"]], left_on="prison_name", right_on="prison_name_original", how="left")
all_data = all_data.fillna(0)
all_data.head(1)

,prison_name,baseline_cna,in_use_cna,operational_capacity,population,perc_pop_to_used_cna,perc_acc_available,date,source,prison_name_original,prison_name_group
0,Altcourse,794,794,1133,1113,1.4,1,2015-02-27,http://www.gov.uk/government/uploads/system/up...,Altcourse,Altcourse


In [4]:
all_data = all_data.drop(["prison_name", "prison_name_original"], axis=1)
all_data.head(2)

,baseline_cna,in_use_cna,operational_capacity,population,perc_pop_to_used_cna,perc_acc_available,date,source,prison_name_group
0,794,794,1133,1113,1.40,1,2015-02-27,http://www.gov.uk/government/uploads/system/up...,Altcourse
1,408,408,400,391,0.96,1,2015-02-27,http://www.gov.uk/government/uploads/system/up...,Ashfield


In [5]:
all_data  = all_data.pivot_table(index=["prison_name_group", "source", "date"], aggfunc="sum", values = ["operational_capacity", "baseline_cna", "in_use_cna","population"]).reset_index()

In [6]:
prisons = pd.read_csv("old_csv_data/all_prison_data_final_manual.csv")
prisons["lat"]= prisons["lat"].fillna(prisons["lat_manual"])
prisons["lng"]= prisons["lng"].fillna(prisons["lng_manual"])

In [7]:
prisons = prisons[["moj_prison_name", "lat", "lng", "nomis_id"]]

In [8]:
final_pop_data = all_data.merge(prisons, left_on="prison_name_group", right_on="moj_prison_name")
final_pop_data.head()

,prison_name_group,source,date,baseline_cna,in_use_cna,operational_capacity,population,moj_prison_name,lat,lng,nomis_id
0,Acklington,10004C44pop_bull_jan_11.txt,2011-01-28,946,946,946,836,Acklington,55.296464,-1.632415,AKI
1,Acklington,10004C6Apop_bull_feb_11.txt,2011-02-25,946,946,946,820,Acklington,55.296464,-1.632415,AKI
2,Acklington,prison-population-bulletin-july-2011.txt,2011-07-29,946,890,890,826,Acklington,55.296464,-1.632415,AKI
3,Acklington,prison-population-bulletin-june.txt,2011-06-24,946,890,890,831,Acklington,55.296464,-1.632415,AKI
4,Acklington,prison-population-bulletin-march-2011.txt,2011-03-25,946,890,890,819,Acklington,55.296464,-1.632415,AKI


In [9]:
final_pop_data["perc_pop_to_used_cna"] = final_pop_data["population"]*1.0/final_pop_data["in_use_cna"]*1.0
final_pop_data["perc_acc_available"] = final_pop_data["in_use_cna"]*1.0/final_pop_data["baseline_cna"]*1.0

In [10]:
# join on meta data
metas = pgs.drop_duplicates("prison_name_group")
metas = metas.drop(["prison_name_original", "prison_meta_info_source", "closed", "done"], axis=1)
final_pop_data = final_pop_data.merge(metas, left_on = "prison_name_group", right_on="prison_name_group", how="left")

In [11]:
final_pop_data = final_pop_data.sort(["moj_prison_name", "date"])
final_pop_data = final_pop_data.drop("prison_name_group", axis=1)
final_pop_data.to_csv("final_full_time_series.csv", index=False)
final_pop_data.head()

/Users/robinlinacre/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,source,date,baseline_cna,in_use_cna,operational_capacity,population,moj_prison_name,lat,lng,nomis_id,...,cat_d,open,male,youth,adult,local,high_security,sex_offender,operator,opened
0,10004C44pop_bull_jan_11.txt,2011-01-28,946,946,946,836,Acklington,55.296464,-1.632415,AKI,...,0,0,1,0,1,0,0,0,HMPS,1972
1,10004C6Apop_bull_feb_11.txt,2011-02-25,946,946,946,820,Acklington,55.296464,-1.632415,AKI,...,0,0,1,0,1,0,0,0,HMPS,1972
4,prison-population-bulletin-march-2011.txt,2011-03-25,946,890,890,819,Acklington,55.296464,-1.632415,AKI,...,0,0,1,0,1,0,0,0,HMPS,1972
6,prison-population-monthly-bulletin-april2011.txt,2011-04-15,946,890,890,851,Acklington,55.296464,-1.632415,AKI,...,0,0,1,0,1,0,0,0,HMPS,1972
5,prison-population-bulletin-may-2011.txt,2011-05-27,946,890,890,841,Acklington,55.296464,-1.632415,AKI,...,0,0,1,0,1,0,0,0,HMPS,1972
